In [1]:
!pip install peft
!pip install trl
!pip install -U transformers
!pip install bitsandbytes
!pip install --upgrade transformers
!pip uninstall autogluon-multimodal autogluon-timeseries -y
!pip install transformers==4.46.0
!pip uninstall -y scikit-learn scipy
!pip install scikit-learn==1.3.0 scipy==1.12.0
!pip install torch
!pip install datasets
!pip install pandas

  Using cached transformers-4.46.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
Using cached transformers-4.46.0-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
Found existing installation: scikit-learn 1.3.0
Uninstalling scikit-learn-1.3.0:
  Successfully uninstalled scikit-learn-1.3.0
Found existing installation: scipy 1.12.0
Uninstalling scipy-1.12.0:
  Suc

In [2]:
!pip install --upgrade transformers
!pip install --upgrade peft

  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.50.3-py3-none-any.whl (10.2 MB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0
    Uninstalling transformers-4.46.0:
      Successfully uninstalled transformers-4.46.0


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
import datasets
import torch
import peft
import trl
import transformers
from huggingface_hub import interpreter_login
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

2025-03-31 05:56:36.949029: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 05:56:36.963561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 05:56:36.982023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 05:56:36.987648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 05:56:37.000611: I tensorflow/core/platform/cpu_feature_guar

In [5]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


In [6]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en', split = 'train')

In [7]:
dataset[0]

{'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. \n\nThe fact that she underwent a Q-tip test is intriguing too. This 

In [8]:
def preprocess_data(example):
    return {
        'input_text': example['Question'] + "\n" + example['Complex_CoT'],
        'labels': example['Response']
    }

In [9]:
train_data = dataset.map(preprocess_data, remove_columns = ['Question', 'Complex_CoT', 'Response'])

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

In [10]:
train_data.to_csv("formated_training_data.csv", index = False)

Creating CSV from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

71592159

In [11]:
dataset = load_dataset("csv", data_files = "formated_training_data.csv", split = 'train')

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
df = pd.DataFrame(dataset)

In [13]:
df.head()

,input_text,labels
0,A 61-year-old woman with a long history of inv...,Cystometry in this case of stress urinary inco...
1,A 45-year-old man with a history of alcohol us...,Considering the clinical presentation of sudde...
2,A 45-year-old man presents with symptoms inclu...,Based on the clinical findings presented—wide-...
3,A patient with psoriasis was treated with syst...,The development of generalized pustules in a p...
4,What is the most likely diagnosis for a 2-year...,Based on the described symptoms and the unusua...


In [14]:
def format_row(row):
    question = row['input_text']
    answer = row['labels']
    formatted_string = f"[INST] {question} [/INST] {answer}"
    return formatted_string

In [15]:
df['text'] = df.apply(format_row, axis = 1)

In [16]:
df['text']

0        [INST] A 61-year-old woman with a long history...
1        [INST] A 45-year-old man with a history of alc...
2        [INST] A 45-year-old man presents with symptom...
3        [INST] A patient with psoriasis was treated wi...
4        [INST] What is the most likely diagnosis for a...
                               ...                        
25366    [INST] Based on the details provided about a 5...
25367    [INST] What medication should be prescribed to...
25368    [INST] A child who experienced disruptions in ...
25369    [INST] An antenna radiates a power of 8 kW and...
25370    [INST] A 67-year-old man with type 2 diabetes ...
Name: text, Length: 25371, dtype: object

In [17]:
train_data = df[['text']]

In [18]:
# Manually split the training data into training and validation sets
train_dataset, val_dataset = train_test_split(train_data, test_size = 0.1, random_state = 42)

In [19]:
train_dataset

,text
1131,[INST] A highway truck driver has profuse rhin...
2265,"[INST] Today, Steven went to the gas station. ..."
2041,[INST] A 30-year-old female patient complains ...
10476,[INST] What is one factor that is not an expla...
21440,"[INST] On the floor, you see a bunch of object..."
...,...
21575,[INST] If a spring is activated by 0.5 mm and ...
5390,"[INST] In the context of a bomb blast injury, ..."
860,[INST] What medication should be administered ...
15795,[INST] A 36-year-old female presents to the cl...


In [20]:
base_model_name = "meta-llama/Llama-3.2-1B"
dataset_name = "FreedomIntelligence/medical-o1-reasoning-SFT"
fine_tuned_model_path = "fine-tuned/Llama-3.2-1B"


# QLORA Parameters
lora_r = 64  # LoRA attention dimention
lora_alpha = 16  # Alpha parameter for LoRA scaling
lora_dropout = 0.1  # Dropout probability for LoRA layers

# Bitsandbytes Parameters
use_4bit = True  # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16"  # Compute dtype for 4-bit base model
bnb_4bit_quant_type = "nf4"  # Quantization type (fp4 or nf4)
use_nested_quant = False  # Activate nested quantization for 4-bit base model (doble quantization)

# Training Parameters
output_dir = "./results"  # Output directory where models prediction checkpoints will be stored
num_train_epochs = 1  # Number of training epochs (no. of time model will see each data point while fine-tuning)
fp16 = True  # fp16-(mixed half-precision) set fp16 to True with a V100 (supports natively)
bf16 = False  # bf16-(brain floating point) set bf16 to True with an A100 (supports natively)
per_device_train_batch_size = 1  # batch size per GPU for training
per_device_eval_batch_Size = 1  # batch size per GPU for evaluation
gradient_accumulation_steps = 4  # no. of update steps to accumulate the gradients for
gradient_checkpointing = True  # Gradient checkpointing 
max_grad_norm = 0.3  # Maximum gradient normal (gradient clipping)
learning_rate = 2e-4  # Initial learning rate (AdamW optimizer)
weight_decay = 0.001  # Weight decay to apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit"  # Optimizer to use
lr_scheduler_type = "cosine"  # Learning rate schedule
max_steps = -1  # Number of training steps (overrides num_train_epochs)
warmup_ratio = 0.03  # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = True  # Group sequences into batches with same length, Saves memory and speeds up training considerably
save_steps = 0  # Save checkpoint every X updates steps
logging_steps = 25  # Log every X updates steps

# SFT parameters
max_seq_length = 512  # Maximum sequence length to use
packing = False  # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0}  # Load the entire model on the GPU 0

In [21]:
# Load dataset (you can process it here)
data = datasets.load_dataset(dataset_name, 'en' ,split="train")

# Select only the first 500 samples
dataset = data.select(range(500))

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [22]:
# Load base model
model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.num_hidden_layers = 12

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [23]:
model_name = "meta-llama/Llama-3.2-1B"
# Load LLaMA tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [24]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_dataset)

In [25]:
train_dataset.save_to_disk("hf_train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/22833 [00:00<?, ? examples/s]

In [26]:
from datasets import load_from_disk
hf_train_dataset = load_from_disk("hf_train_dataset")

In [27]:
hf_train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 22833
})

In [28]:
def tokenize_function(examples):  # Handle batched inputs
    tokens = tokenizer(
        examples["text"],  # Pass list of texts
        truncation=True,
        padding="longest"
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Labels = input_ids
    return tokens

In [29]:
tokenized_train_dataset = hf_train_dataset.map(
    tokenize_function, 
    batched=True, 
    batch_size=1  # Adjust based on memory availability
)

Map:   0%|          | 0/22833 [00:00<?, ? examples/s]

In [30]:
tokenized_train_dataset.save_to_disk("tokenized_train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/22833 [00:00<?, ? examples/s]

In [31]:
from datasets import load_from_disk
tokenized_train_dataset = load_from_disk("tokenized_train_dataset")

In [32]:
tokenized_train_dataset.features

{'text': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [33]:
# Load LoRA configuration
peft_config = peft.LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = transformers.TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = trl.SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    peft_config=peft_config,
    args=training_arguments
)

Truncating train dataset:   0%|          | 0/22833 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [34]:
trainer.train()

Step,Training Loss
25,7.683300
50,7.392300
75,5.679700
100,4.539100
125,4.195200
150,3.802000
175,3.747400
200,3.603800
225,3.625800
250,3.394900


TrainOutput(global_step=5708, training_loss=2.849515317116238, metrics={'train_runtime': 5902.8829, 'train_samples_per_second': 3.868, 'train_steps_per_second': 0.967, 'total_flos': 8.303527453796352e+16, 'train_loss': 2.849515317116238})

In [35]:
# Save trained model
trainer.model.save_pretrained(fine_tuned_model_path)

In [50]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

In [51]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load fine-tuned LoRA/PEFT adapter
model = PeftModel.from_pretrained(model, fine_tuned_model_path)

In [53]:
prompt = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

# Create the text-generation pipeline with truncation explicitly enabled
pipe = transformers.pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
result = pipe(f"<s>[INST] {prompt} [/INST]")

# Print the generated text
result[0]['generated_text']

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

'<s>[INST] A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions? [/INST] Cystometry would most likely reveal that the patient has a normal residual volume. Cystometry measures the amount of urine that is retained after you empty the bladder. This would be consistent with her symptoms, which are consistent with a urinary catheter. In the context of a urinary catheter, the residual volume is the amount of urine that is still in the bladder. This is because the catheter is not removing the urine. The detrusor contractions would not be seen in this scenario. The residual volume is not a contraction. This is because the bladder is empty. The detrusor contraction is a muscle contraction that increases the'

In [ ]:
# Push model to hugging face
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("Vaibhavkr007/Llama-2-7b-hf-finetune", check_pr=True)
tokenizer.push_to_hub("Vaibhavkr007/Llama-2-7b-hf-finetune",check_pr=True)

In [38]:
# Empty VRAM
# del model
# del pipe
# del trainer
import gc
gc.collect()
gc.collect()

0

In [39]:
import os
import boto3

In [45]:
s3_bucket_name = "aws-bucket-0710"  # Replace with your bucket name
s3_model_prefix = "fine-tuned-model"  # Prefix for model in S3
model_local_dir = "fine-tuned/Llama-3.2-1B"

In [47]:
s3_client = boto3.client('s3')
for root, _, files in os.walk(model_local_dir):
    for file in files:
        local_path = os.path.join(root, file)
        relative_path = os.path.relpath(local_path, model_local_dir)
        s3_path = os.path.join(s3_model_prefix, relative_path)
        s3_client.upload_file(local_path, s3_bucket_name, s3_path)
        print(f"Uploaded {file} to s3://{s3_bucket_name}/{s3_path}")

Uploaded README.md to s3://aws-bucket-0710/fine-tuned-model/README.md
Uploaded adapter_model.safetensors to s3://aws-bucket-0710/fine-tuned-model/adapter_model.safetensors
Uploaded adapter_config.json to s3://aws-bucket-0710/fine-tuned-model/adapter_config.json


In [49]:
s3_client.list_objects_v2(Bucket = s3_bucket_name, Prefix="fine-tuned-model/")

{'ResponseMetadata': {'RequestId': 'E4R9XW3BHWHWJJ65',
  'HostId': 'p9sVo7eK+NT7vIRG9ZcQQICbRsngkVq53YEkKBAz3u6UMa/jqFe124UnnJ7HXKp2vpMxthmn27s=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'p9sVo7eK+NT7vIRG9ZcQQICbRsngkVq53YEkKBAz3u6UMa/jqFe124UnnJ7HXKp2vpMxthmn27s=',
   'x-amz-request-id': 'E4R9XW3BHWHWJJ65',
   'date': 'Mon, 31 Mar 2025 09:46:21 GMT',
   'x-amz-bucket-region': 'eu-north-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'fine-tuned-model/README.md',
   'LastModified': datetime.datetime(2025, 3, 31, 9, 45, 48, tzinfo=tzlocal()),
   'ETag': '"c2018900850155d0bf0d98036ee8f7f3"',
   'ChecksumAlgorithm': ['CRC32'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 5097,
   'StorageClass': 'STANDARD'},
  {'Key': 'fine-tuned-model/adapter_config.json',
   'LastModified': datetime.datetime(2025, 3, 31, 9, 45, 50, tzinfo=tzlocal()),
   'ETag': '"3c